In [1]:
import pandas as pd
import json
from datetime import datetime
from random import randint
def mask(df, key, value):
    return df[df[key] == value]
pd.DataFrame.mask = mask

In [2]:
class AdSlots():
    total = 100
    slots = 3

In [33]:
class User(object):
    def __init__(self, request):
        # request ==> { 'query':'Samsung TV', 'query_type':'Television', 'adslots':Object::AdSlots }
        assert type(request) == dict
        self.request = request
        self.req_id = randint(1000000000, 1999999999)
        self.time = datetime.now()
    def get_response(self):
        # in here we create a response object and return it to the respective function call
        auction = Auction(self.request)
        auction.targeting()
        auction.run_auction()
        return auction
    def get_targeting(self):
        # here we return the targeting clauses for the respective class of query such as televisions , utensils etc ....
        # returns a list of targeting clauses extracted from a csv file for the respective query_type
        with open('ads.json') as ads:
            ad_data = json.load(ads)
        targeting_clauses = []
        for i in ad_data[self.request["query_type"]]:
            targeting_clauses.extend(list(j.lower() for j in i["matchClauses"]))
        
        return targeting_clauses

class Auction(object):
    def __init__(self, request):
        self.request = User(request)
        self.candidates = self.get_auction_participants()
        self.targeting_clauses = self.request.get_targeting()
        self.positives = dict(zip(self.targeting_clauses, [0]*len(self.targeting_clauses)))
        self.winners = {}
    def targeting(self):
        for i in self.targeting_clauses:
            self.get_(self.request.request['query'], i)
        print(f"Targeting clauses filtered with the following response ==> \n`{str(self.positives)}`")
    def get_(self, q, tc):
        # comparision of the query and the targeting clause goes here 
        # A mock of switch case goes here
        q = q.lower()
        if q == tc or q in tc:
            self.positives[tc] = 1
        elif self.request.request['query_type']:
            self.positives[tc] = -1
        else:
            pass
    def get_auction_participants(self):
        total = pd.read_csv('auction-participants.csv')
#         print(total.mask('type', " "+self.request.request['query_type']))
        auc_participants = total.mask('type', " "+self.request.request['query_type'])
        return auc_participants
    def run_auction(self):
        with open('ads.json') as ads:
            ad_data = json.load(ads)

        for i in self.positives.keys():
#             print(self.positives)
            if self.positives[i] != 0 and self.positives[i] == 1:
                # the given query has a perfect match within the db
                for j in ad_data[self.request.request['query_type']]:
                    dat = list(x.lower() for x in j['matchClauses'])
                    if i in dat:
                        try:
                            self.winners[j['campId']] = int(self.candidates.mask('campaign_id', int(j['campId']))['bid'])
                        except Exception as e:
                            pass
            # broad will be written soon
        
        let = sorted(self.winners, key=self.winners.get)[::-1][0:self.request.request['adslots'].slots]
        self.winners = {i:self.winners[i] for i in let }


In [34]:
class Campaigns(object):
    def __init__(self, cid, budget):
        self.cid = cid
        self.budget = budget

In [35]:
adslots = AdSlots()
query = {'query': 'TV', 'query_type': "Television", 'adslots':adslots}
user = User(query)

In [36]:
auction = user.get_response()

Targeting clauses filtered with the following response ==> 
`{'sanyo': -1, 'sanyo tv': 1, 'sanya tv': 1, 'sanyo 32 inch tv': 1, 'lg': -1, 'lg tv': 1, 'lg 32 inch tv': 1}`


In [37]:
auction.winners

{1002: 12000, 1000: 9000, 1005: 8000}